### Correlation
Convolution is a filtering operation that expresses the amount of overlap of one function as it is shifted over another function.
$$f \otimes h = \sum_k\sum_l f(k, l)h(k, l)$$

### Convolution
Correlation compares the similarity of two sets of data
$$f * h = \sum_k\sum_l f(k, l)h(-k, -l)$$

### Gaussian filter
Used for blurring images
- 1D: $g(x)=e^\frac{-x^2}{2\sigma ^2}$
- 2D: $g(x, y)=e^\frac{-(x^2 + y^2)}{2\sigma ^2}$

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def gaussian_blur(image_tensor: torch.Tensor, kernel_size=5, sigma=1.0):
    channels, height, width = image_tensor.size()

    # Create 2D Gaussian kernel
    grid = torch.linspace(-kernel_size // 2 + 1, kernel_size // 2, kernel_size)
    gaussian_kernel = torch.exp(-((grid[:, None] ** 2 + grid[None, :] ** 2) / (2 * sigma ** 2)))
    gaussian_kernel /= gaussian_kernel.sum()

    # Create a 4D weight tensor for convolution
    gaussian_kernel = gaussian_kernel.expand(channels, 1, kernel_size, kernel_size)
    weight = gaussian_kernel.to(device)

    # Apply Gaussian blur
    image_tensor = image_tensor.unsqueeze(0)
    blurred_image = F.conv2d(image_tensor, weight, padding=kernel_size // 2, groups=channels)
    return blurred_image.squeeze(0)


In [ ]:
image = Image.open("img1.png")
image_tensor = transforms.ToTensor()(image).to(device)
blurred_image_tensor = gaussian_blur(image_tensor)

blurred_image = transforms.ToPILImage()(blurred_image_tensor)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].imshow(image)
axes[1].imshow(blurred_image)

axes[0].axis('off')
axes[1].axis('off')

plt.show()